# MODEL

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches
import logging
import os
from ipywidgets import interact, interactive
from IPython.display import HTML

# Configuration constants
FIGURE_SIZE = (5, 4)
THUMBNAIL_SIZE = (120, 120)
POSITION_MAP = {
    "top-left": (0, 0),
    "top-right": (0, 1),
    "bottom-left": (1, 0),
    "bottom-right": (1, 1)
}

# Setup logging
logging.basicConfig(level=logging.INFO)

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Utility function to transform delta list
def transform_delta_list(delta_list):
    transformed_list = []
    for delta in delta_list:
        if delta <= 50:
            transformed_list.append(f"{delta}dc")
        elif delta >= 50:
            transformed_list.append(f"{100 - delta}dp")
    return transformed_list

# Step 2: Widget Manager Class
class WidgetManager:
    def __init__(self, df):
        # Define the options for call and put level widgets using transform_delta_list
        delta_list = [50, 45, 40, 35, 25, 15, 10, 5, 55, 60, 65, 75, 85, 90, 95]
        transformed_delta_list = transform_delta_list(delta_list)
        call_level_widget_options = [(delta, delta) for delta in transformed_delta_list if "dc" in delta]
        put_level_widget_options = [(delta, delta) for delta in transformed_delta_list if "dp" in delta]

        # Preset widgets to match screenshot configuration
        self.plot_type_widget = widgets.Dropdown(options=['Time Series Plot', 'Custom Stats Chart'], value='Time Series Plot', description="Plot Type:", layout=widgets.Layout(width='100%'))
        self.udl_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('UDL').unique(),
            value=['US_SPX'],  # Preset UDL as per screenshot
            description='UDL:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.type_widget = widgets.Dropdown(
            options=['Spot', 'Moneyness', 'Delta'],
            value='Moneyness',  # Preset Type as per screenshot
            description='Type:',
            layout=widgets.Layout(width='100%')
        )
        self.param_widget = widgets.Dropdown(
            options=['Level', 'Returns'],
            value='Level',  # Preset Param as per screenshot
            description='Param:',
            layout=widgets.Layout(width='100%')
        )
        self.matu_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Matu').unique(),
            value=[3],  # Preset Maturity as per screenshot
            description='Maturity:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.level_widget = widgets.SelectMultiple(
            options=[str(m) for m in df.columns.get_level_values('Level').unique()],  # Ensure widget options are in sync with generated DataFrame
            value=[str(df.columns.get_level_values('Level').unique()[0])],  # Default to the first available level
            description='Level:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.put_level_widget = widgets.SelectMultiple(
            options=put_level_widget_options,
            value=[put_level_widget_options[0][0]] if put_level_widget_options else [],
            description='Put Level:',
            layout=widgets.Layout(height='100px', width='100%', display='none')
        )
        self.call_level_widget = widgets.SelectMultiple(
            options=call_level_widget_options,
            value=[call_level_widget_options[0][0]] if call_level_widget_options else [],
            description='Call Level:',
            layout=widgets.Layout(height='100px', width='100%', display='none')
        )
        self.normalised_checkbox = widgets.Checkbox(
            value=False,
            description='Normalised',
            layout=widgets.Layout(width='100%', display='none')
        )
        self.window_widget = widgets.Dropdown(options=[5, 10, 20, 52, 104, 156], value=5, description='Window:', layout=widgets.Layout(height='40px', width='100%'))
        self.start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp('2023-01-01').to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp('2024-11-18').to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.plot_button = widgets.Button(description='Preview Plot', button_style='success', layout=widgets.Layout(width='28%', min_width='120px'))
        self.add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True, layout=widgets.Layout(width='28%', min_width='120px'))
        self.export_button = widgets.Button(description="Export to PPT", button_style='warning', layout=widgets.Layout(width='28%', min_width='120px'))
        self.position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:", layout=widgets.Layout(height='40px', width='100%'))
        self.add_slide_button = widgets.Button(description="Add Slide", button_style="primary", layout=widgets.Layout(width='28%', min_width='120px'))

# Step 3: Plot Manager to Register Plots and Manage Widgets
class PlotManager:
    def __init__(self):
        self.plots = {}

    def register_plot(self, name, plot_function, required_widgets):
        """Register a new plot type."""
        self.plots[name] = {
            "function": plot_function,
            "widgets": required_widgets
        }

    def get_plot_function(self, name):
        return self.plots.get(name, {}).get("function")

    def get_required_widgets(self, name):
        return self.plots.get(name, {}).get("widgets", [])

# Step 4: Registering Plot Functions
def create_time_series_plot(filtered_df, window):
    result_df = filtered_df.rolling(window=window).mean().fillna(0)
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    for column in result_df.columns:
        ax.plot(result_df.index, result_df[column], label=str(column))
    ax.set_title("Time Series Plot Preview")
    ax.legend()
    return fig

def create_stats_chart(filtered_df):
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    categories = filtered_df.columns.get_level_values('UDL').unique()
    min_values = filtered_df.min().values
    max_values = filtered_df.max().values
    avg_values = filtered_df.mean().values
    last_values = filtered_df.iloc[-1].values
    percentile_20 = filtered_df.quantile(0.2).values
    percentile_80 = filtered_df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    return fig

# Step 5: Utility function to generate plot image
def generate_plot_image(fig, ax):
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_stream.seek(0)
    img = Image.open(img_stream)
    ax.imshow(img, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

# Step 6: Slide management class
class SlideManager:
    def __init__(self):
        self.slides = [{"plots": []}]
        self.selected_slide_index = 0

    def add_slide(self):
        self.slides.append({"plots": []})
        self.selected_slide_index = len(self.slides) - 1

    def add_plot_to_slide(self, plot_data, position):
        slide_index = self.selected_slide_index
        # Remove any existing plot in the selected position
        self.slides[slide_index]["plots"] = [plot for plot in self.slides[slide_index]["plots"] if plot["position"] != position]
        # Add the new plot to the selected position
        self.slides[slide_index]["plots"].append(plot_data)

    def get_current_slide(self):
        return self.slides[self.selected_slide_index]

# Step 7: Main app class
class App:
    def __init__(self, df):
        # Adjust df_below_output to tightly fit the main display without extra white space
        self.df_below_output = widgets.Output(layout=widgets.Layout(width="100%", border="1px solid #d3d3d3", padding="10px", margin="0 auto"))  # Adjusted output view for filtered DataFrame
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.register_plots()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="70%", height="500px", border="1px solid black", padding="0px", margin="0 auto"))  # Adjusted to match the width with df_below_output
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.input_changes_dict = {}  # Dictionary to track input changes
        self.filtered_df = df  # Initialize with the full DataFrame
        self.build_layout()
        self.bind_events()
        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def register_plots(self):
        self.plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
        self.plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "start_date_widget", "end_date_widget"])

        # Update plot_type_widget with available plot types
        self.widgets.plot_type_widget.options = list(self.plot_manager.plots.keys())
        self.widgets.plot_type_widget.value = list(self.plot_manager.plots.keys())[0]  # Set default value

    def build_layout(self):
        control_buttons = widgets.GridBox([
            self.widgets.add_slide_button, self.widgets.export_button,
            self.widgets.plot_button, self.widgets.add_to_slide_button
        ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
        
        self.customization_window = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
            self.widgets.plot_type_widget,
            self.widgets.udl_widget,
            self.widgets.type_widget,
            self.widgets.param_widget,
            self.widgets.level_widget,
            self.widgets.put_level_widget,
            self.widgets.call_level_widget,
            self.widgets.window_widget,
            self.widgets.start_date_widget,
            self.widgets.end_date_widget,
            self.widgets.position_dropdown,
            control_buttons,
        ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
        
        # Display customization window on the right, plot or slide view in the center
        display(widgets.VBox([
            widgets.HBox([self.left_sidebar, self.main_display, self.customization_window], layout=widgets.Layout(align_items='flex-start')),
            self.df_below_output  # Displaying the filtered DataFrame below
        ]))

    def display_data_below(self, df):
        # Display the filtered dataframe below the main view without clearing the existing content
        with self.df_below_output:
            clear_output(wait=True)
            display(HTML(df.to_html(max_rows=10)))  # Set the table width to "100%" so it fits within its container

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.type_widget.observe(self.on_type_change, names='value')
        self.widgets.param_widget.observe(self.on_param_change, names='value')
        self.widgets.matu_widget.observe(self.on_widget_change, names='value')
        self.widgets.udl_widget.observe(self.on_widget_change, names='value')
        self.widgets.level_widget.observe(self.on_widget_change, names='value')
        self.widgets.plot_button.on_click(self.on_plot_button_clicked)
        self.widgets.add_to_slide_button.on_click(self.on_add_to_slide_button_clicked)
        self.widgets.add_slide_button.on_click(self.on_add_slide_button_clicked)
        self.widgets.export_button.on_click(self.on_export_button_clicked)

    def on_type_change(self, change):
        selected_type = change['new']
        if selected_type == 'Spot':
            self.widgets.param_widget.options = ['Level', 'Returns']
            self.widgets.param_widget.value = 'Level'
            # Hide Maturity and Level-related widgets for Spot with Level selection
            self.widgets.matu_widget.layout.display = 'none'
            self.widgets.level_widget.layout.display = 'none'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
        elif selected_type in ['Moneyness', 'Delta']:
            self.widgets.param_widget.options = ['Level', 'Carry', 'Convex', 'Shift', 'Skew', 'Term Structure']
            self.widgets.param_widget.value = 'Level'
            # Show Maturity and Level-related widgets for Moneyness and Delta
            self.widgets.matu_widget.layout.display = 'flex'
            self.widgets.level_widget.layout.display = 'flex'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
            
            # Update level options based on selected type
            if selected_type == 'Delta':
                self.widgets.level_widget.options = [str(delta) for delta in [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95]]
            elif selected_type == 'Moneyness':
                self.widgets.level_widget.options = [str(m) for m in self.df.columns.get_level_values('Level').unique()]
        
        # Hide/show widgets depending on selected param
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def on_param_change(self, change):
        # Update visibility of widgets when param changes
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def update_param_widgets_visibility(self):
        param_value = self.widgets.param_widget.value
        self.widgets.window_widget.layout.display = 'none'
        self.widgets.put_level_widget.layout.display = 'none'
        self.widgets.call_level_widget.layout.display = 'none'
        self.widgets.level_widget.layout.display = 'none'
        self.widgets.matu_widget.layout.display = 'none'

        if param_value == 'Returns':
            self.widgets.window_widget.layout.display = 'flex'
        elif param_value in ['Skew', 'Convex']:
            self.widgets.put_level_widget.layout.display = 'flex'
            self.widgets.call_level_widget.layout.display = 'flex'
        elif param_value == 'Level':
            if self.widgets.type_widget.value != 'Spot':
                self.widgets.matu_widget.layout.display = 'flex'
                self.widgets.level_widget.layout.display = 'flex'

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_manager.get_required_widgets(selected_plot)

        for widget_name in ["plot_type_widget", "udl_widget", "matu_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
            widget = getattr(self.widgets, widget_name)
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True
        self.update_filtered_df()

    def on_widget_change(self, change):
        # Update filtered DataFrame whenever a widget value changes
        self.update_filtered_df()

    def update_filtered_df(self):
        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        self.filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
        self.display_data_below(self.filtered_df)

    def filter_data(self, df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date):
        # Ensure that selected_level contains valid entries for filtering
        if selected_param == 'Level' and not selected_level:
            selected_level = df.columns.get_level_values('Level').unique()
        filtered_df = df.loc[start_date:end_date, \
                             (df.columns.get_level_values('UDL').isin(selected_udl)) &
                             (df.columns.get_level_values('Matu').isin(selected_matu)) &
                             (df.columns.get_level_values('Param').isin([selected_type])) &
                             (df.columns.get_level_values('Level').isin(selected_level))]
        return filtered_df

    def on_plot_button_clicked(self, b):
        plot_type = self.widgets.plot_type_widget.value
        plot_function = self.plot_manager.get_plot_function(plot_type)

        # Handle None plot function (invalid plot type)
        if plot_function is None:
            with self.output_plot:
                clear_output(wait=True)
                print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
            return

        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        window = self.widgets.window_widget.value
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        # Track input changes
        self.input_changes_dict = {
            "Plot Type": plot_type,
            "UDL": selected_udl,
            "Maturity": selected_matu,
            "Type": selected_type,
            "Param": selected_param,
            "Level": selected_level,
            "Window": window,
            "Start Date": start_date,
            "End Date": end_date
        }

        # Display debug table below
        with self.df_below_output:
            clear_output(wait=True)
            debug_df = pd.DataFrame(self.input_changes_dict.items(), columns=["Parameter", "Value"])
            display(HTML(debug_df.to_html(index=False)))

        if start_date > end_date:
            with self.output_plot:
                clear_output(wait=True)
                print("Start Date must be before End Date. Please correct your selection.")
            return

        try:
            filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
            if filtered_df.empty:
                with self.output_plot:
                    clear_output(wait=True)
                    print("No valid data available for the selected combination. Please adjust your selection.")
                    self.widgets.add_to_slide_button.disabled = True
                    return

            # Display the filtered DataFrame
            self.display_data_below(filtered_df)

            with self.main_display:
                clear_output(wait=True)
                if window and plot_type == "Time Series Plot":
                    fig = plot_function(filtered_df, window)
                else:
                    fig = plot_function(filtered_df)
                plt.show()
                self.current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": plot_type,
                    "position": self.widgets.position_dropdown.value
                }
                self.widgets.add_to_slide_button.disabled = False
        except Exception as e:
            with self.output_plot:
                clear_output(wait=True)
                print(f"An error occurred while generating the plot: {e}")
                logging.error("Error in on_plot_button_clicked", exc_info=True)

    def on_add_to_slide_button_clicked(self, b):
        if not self.current_plot_data:
            return
        
        # Re-fetch the currently selected position to ensure the latest value is used
        position = self.widgets.position_dropdown.value
        self.current_plot_data["position"] = position
        
        slide = self.slide_manager.get_current_slide()
        
        # Validate position is not already occupied
        if any(plot["position"] == position for plot in slide["plots"]):
            with self.main_display:
                clear_output(wait=True)
                print(f"Position '{position}' is already occupied. Please choose another position.")
                return

        # Add the plot to the slide if position is available
        self.slide_manager.add_plot_to_slide(self.current_plot_data.copy(), position)
        self.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
        self.switch_to_slide_view()
        self.update_sidebar()

    def on_add_slide_button_clicked(self, b):
        self.slide_manager.add_slide()
        self.update_sidebar()
        self.switch_to_slide_view()

    def on_export_button_clicked(self, b):
        prs = Presentation()
        for slide in self.slide_manager.slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig = plot_data["figure"]
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)
                left, top = self.get_slide_position(plot_data["position"])
                slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        pptx_filename = "Generated_Presentation.pptx"
        prs.save(pptx_filename)
        logging.info(f"Presentation exported as '{pptx_filename}'")
        
        # Ensure file visibility in Jupyter environment
        if os.path.exists(pptx_filename):
            print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
        else:
            print("Error: The presentation could not be saved.")

    def get_slide_position(self, position):
        if position == "top-left":
            return Inches(0), Inches(0)
        elif position == "top-right":
            return Inches(5), Inches(0)
        elif position == "bottom-left":
            return Inches(0), Inches(3.75)
        elif position == "bottom-right":
            return Inches(5), Inches(3.75)
        return Inches(0), Inches(0)

    def switch_to_slide_view(self):
        self.main_display.clear_output(wait=True)
        with self.main_display:
            slide = self.slide_manager.get_current_slide()
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)
                    ax.set_title(plot_data["type"])
                else:
                    logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

            plt.tight_layout()
            plt.show()

    def update_sidebar(self):
        sidebar_content = []
        for i, slide in enumerate(self.slide_manager.slides):
            fig, axs = plt.subplots(2, 2, figsize=(4, 4))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)

            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            plt.close(fig)
            buf.seek(0)
            img = Image.open(buf)
            img.thumbnail(THUMBNAIL_SIZE)

            with BytesIO() as output:
                img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
            slide_button_box.on_click(lambda b, idx=i: self.select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        self.left_sidebar.children = sidebar_content

    def select_slide(self, index):
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# Step 8: Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': [],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
}

df = create_multi_index_dataframe(udl, matu, param, level)
app = App(df)
display(app)

# CHAP GPT PROCESS 
 
# Use Inline Editing Features: 
The Canvas interface offers inline editing capabilities. Highlight the specific code section you wish to change and provide your correction directly. This method ensures that the system applies your modifications accurately to the intended code block.

# 'Regenerate' Function: prompts system to reproces, incorporating  latest changes - use 'Regenerate' or 'Refresh'  available in interface
This action prompts the system to reprocess the code, incorporating the latest changes.

# Maintain a Sequential Workflow: Ensure that each correction is applied and reflected in the code pane before proceeding to the next.
This step-by-step approach helps prevent the system from overlooking updates due to overlapping instructions.

# PY

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches
import logging
import os
from ipywidgets import interact, interactive
from IPython.display import HTML

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Utility function to transform delta list
def transform_delta_list(delta_list):
    transformed_list = []
    for delta in delta_list:
        if delta <= 50:
            transformed_list.append(f"{delta}dc")
        elif delta >= 50:
            transformed_list.append(f"{100 - delta}dp")
    return transformed_list

# Step 2: Widget Manager Class
class WidgetManager:
    def __init__(self, df):
        # Define the options for call and put level widgets using transform_delta_list
        delta_list = [50, 45, 40, 35, 25, 15, 10, 5, 55, 60, 65, 75, 85, 90, 95]
        transformed_delta_list = transform_delta_list(delta_list)
        call_level_widget_options = [(delta, delta) for delta in transformed_delta_list if "dc" in delta]
        put_level_widget_options = [(delta, delta) for delta in transformed_delta_list if "dp" in delta]

        # Preset widgets to match screenshot configuration
        self.plot_type_widget = widgets.Dropdown(options=['Time Series Plot', 'Custom Stats Chart'], value='Time Series Plot', description="Plot Type:", layout=widgets.Layout(width='100%'))
        self.udl_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('UDL').unique(),
            value=['US_SPX'],  # Preset UDL as per screenshot
            description='UDL:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.type_widget = widgets.Dropdown(
            options=['Spot', 'Moneyness', 'Delta'],
            value='Moneyness',  # Preset Type as per screenshot
            description='Type:',
            layout=widgets.Layout(width='100%')
        )
        self.param_widget = widgets.Dropdown(
            options=['Level', 'Returns'],
            value='Level',  # Preset Param as per screenshot
            description='Param:',
            layout=widgets.Layout(width='100%')
        )
        self.matu_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Matu').unique(),
            value=[3],  # Preset Maturity as per screenshot
            description='Maturity:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.level_widget = widgets.SelectMultiple(
            options=[str(m) for m in df.columns.get_level_values('Level').unique()],  # Ensure widget options are in sync with generated DataFrame
            value=[str(df.columns.get_level_values('Level').unique()[0])],  # Default to the first available level
            description='Level:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.put_level_widget = widgets.SelectMultiple(
            options=put_level_widget_options,
            value=[put_level_widget_options[0][0]] if put_level_widget_options else [],
            description='Put Level:',
            layout=widgets.Layout(height='100px', width='100%', display='none')
        )
        self.call_level_widget = widgets.SelectMultiple(
            options=call_level_widget_options,
            value=[call_level_widget_options[0][0]] if call_level_widget_options else [],
            description='Call Level:',
            layout=widgets.Layout(height='100px', width='100%', display='none')
        )
        self.normalised_checkbox = widgets.Checkbox(
            value=False,
            description='Normalised',
            layout=widgets.Layout(width='100%', display='none')
        )
        self.window_widget = widgets.Dropdown(options=[5, 10, 20, 52, 104, 156], value=5, description='Window:', layout=widgets.Layout(height='40px', width='100%'))
        self.start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp('2023-01-01').to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp('2024-11-18').to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.plot_button = widgets.Button(description='Preview Plot', button_style='success', layout=widgets.Layout(width='28%', min_width='120px'))
        self.add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True, layout=widgets.Layout(width='28%', min_width='120px'))
        self.export_button = widgets.Button(description="Export to PPT", button_style='warning', layout=widgets.Layout(width='28%', min_width='120px'))
        self.position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:", layout=widgets.Layout(height='40px', width='100%'))
        self.add_slide_button = widgets.Button(description="Add Slide", button_style="primary", layout=widgets.Layout(width='28%', min_width='120px'))

# Step 3: Plot Manager to Register Plots and Manage Widgets
class PlotManager:
    def __init__(self):
        self.plots = {}

    def register_plot(self, name, plot_function, required_widgets):
        """Register a new plot type."""
        self.plots[name] = {
            "function": plot_function,
            "widgets": required_widgets
        }

    def get_plot_function(self, name):
        return self.plots.get(name, {}).get("function")

    def get_required_widgets(self, name):
        return self.plots.get(name, {}).get("widgets", [])

# Step 4: Registering Plot Functions
def create_time_series_plot(filtered_df, window):
    result_df = filtered_df.rolling(window=window).mean().fillna(0)
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    for column in result_df.columns:
        ax.plot(result_df.index, result_df[column], label=str(column))
    ax.set_title("Time Series Plot Preview")
    ax.legend()
    return fig

def create_stats_chart(filtered_df):
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    categories = filtered_df.columns.get_level_values('UDL').unique()
    min_values = filtered_df.min().values
    max_values = filtered_df.max().values
    avg_values = filtered_df.mean().values
    last_values = filtered_df.iloc[-1].values
    percentile_20 = filtered_df.quantile(0.2).values
    percentile_80 = filtered_df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    return fig

# Step 5: Utility function to generate plot image
def generate_plot_image(fig, ax):
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_stream.seek(0)
    img = Image.open(img_stream)
    ax.imshow(img, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

# Step 6: Slide management class
class SlideManager:
    def __init__(self):
        self.slides = [{"plots": []}]
        self.selected_slide_index = 0

    def add_slide(self):
        self.slides.append({"plots": []})
        self.selected_slide_index = len(self.slides) - 1

    def add_plot_to_slide(self, plot_data, position):
        slide_index = self.selected_slide_index
        # Remove any existing plot in the selected position
        self.slides[slide_index]["plots"] = [plot for plot in self.slides[slide_index]["plots"] if plot["position"] != position]
        # Add the new plot to the selected position
        self.slides[slide_index]["plots"].append(plot_data)

    def get_current_slide(self):
        return self.slides[self.selected_slide_index]

# Step 1: Move All Methods Out of the Class App - OK
# APP - COPY

In [3]:
# Step 7: Main app class
class App:
    def __init__(self, df):
        # Adjust df_below_output to tightly fit the main display without extra white space
        self.df_below_output = widgets.Output(layout=widgets.Layout(width="100%", border="1px solid #d3d3d3", padding="10px", margin="0 auto"))  # Adjusted output view for filtered DataFrame
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.register_plots()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="70%", height="500px", border="1px solid black", padding="0px", margin="0 auto"))  # Adjusted to match the width with df_below_output
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.input_changes_dict = {}  # Dictionary to track input changes
        self.filtered_df = df  # Initialize with the full DataFrame
        self.build_layout()
        self.bind_events()
        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def register_plots(self):
        self.plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
        self.plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "start_date_widget", "end_date_widget"])

        # Update plot_type_widget with available plot types
        self.widgets.plot_type_widget.options = list(self.plot_manager.plots.keys())
        self.widgets.plot_type_widget.value = list(self.plot_manager.plots.keys())[0]  # Set default value

    def build_layout(self):
        control_buttons = widgets.GridBox([
            self.widgets.add_slide_button, self.widgets.export_button,
            self.widgets.plot_button, self.widgets.add_to_slide_button
        ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
        
        self.customization_window = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
            self.widgets.plot_type_widget,
            self.widgets.udl_widget,
            self.widgets.type_widget,
            self.widgets.param_widget,
            self.widgets.level_widget,
            self.widgets.put_level_widget,
            self.widgets.call_level_widget,
            self.widgets.window_widget,
            self.widgets.start_date_widget,
            self.widgets.end_date_widget,
            self.widgets.position_dropdown,
            control_buttons,
        ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
        
        # Display customization window on the right, plot or slide view in the center
        display(widgets.VBox([
            widgets.HBox([self.left_sidebar, self.main_display, self.customization_window], layout=widgets.Layout(align_items='flex-start')),
            self.df_below_output  # Displaying the filtered DataFrame below
        ]))

    def display_data_below(self, df):
        # Display the filtered dataframe below the main view without clearing the existing content
        with self.df_below_output:
            clear_output(wait=True)
            display(HTML(df.to_html(max_rows=10)))  # Set the table width to "100%" so it fits within its container

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.type_widget.observe(self.on_type_change, names='value')
        self.widgets.param_widget.observe(self.on_param_change, names='value')
        self.widgets.matu_widget.observe(self.on_widget_change, names='value')
        self.widgets.udl_widget.observe(self.on_widget_change, names='value')
        self.widgets.level_widget.observe(self.on_widget_change, names='value')
        self.widgets.plot_button.on_click(self.on_plot_button_clicked)
        self.widgets.add_to_slide_button.on_click(self.on_add_to_slide_button_clicked)
        self.widgets.add_slide_button.on_click(self.on_add_slide_button_clicked)
        self.widgets.export_button.on_click(self.on_export_button_clicked)

    def on_type_change(self, change):
        selected_type = change['new']
        if selected_type == 'Spot':
            self.widgets.param_widget.options = ['Level', 'Returns']
            self.widgets.param_widget.value = 'Level'
            # Hide Maturity and Level-related widgets for Spot with Level selection
            self.widgets.matu_widget.layout.display = 'none'
            self.widgets.level_widget.layout.display = 'none'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
        elif selected_type in ['Moneyness', 'Delta']:
            self.widgets.param_widget.options = ['Level', 'Carry', 'Convex', 'Shift', 'Skew', 'Term Structure']
            self.widgets.param_widget.value = 'Level'
            # Show Maturity and Level-related widgets for Moneyness and Delta
            self.widgets.matu_widget.layout.display = 'flex'
            self.widgets.level_widget.layout.display = 'flex'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
            
            # Update level options based on selected type
            if selected_type == 'Delta':
                self.widgets.level_widget.options = [str(delta) for delta in [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95]]
            elif selected_type == 'Moneyness':
                self.widgets.level_widget.options = [str(m) for m in self.df.columns.get_level_values('Level').unique()]
        
        # Hide/show widgets depending on selected param
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def on_param_change(self, change):
        # Update visibility of widgets when param changes
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def update_param_widgets_visibility(self):
        param_value = self.widgets.param_widget.value
        self.widgets.window_widget.layout.display = 'none'
        self.widgets.put_level_widget.layout.display = 'none'
        self.widgets.call_level_widget.layout.display = 'none'
        self.widgets.level_widget.layout.display = 'none'
        self.widgets.matu_widget.layout.display = 'none'

        if param_value == 'Returns':
            self.widgets.window_widget.layout.display = 'flex'
        elif param_value in ['Skew', 'Convex']:
            self.widgets.put_level_widget.layout.display = 'flex'
            self.widgets.call_level_widget.layout.display = 'flex'
        elif param_value == 'Level':
            if self.widgets.type_widget.value != 'Spot':
                self.widgets.matu_widget.layout.display = 'flex'
                self.widgets.level_widget.layout.display = 'flex'

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_manager.get_required_widgets(selected_plot)

        for widget_name in ["plot_type_widget", "udl_widget", "matu_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
            widget = getattr(self.widgets, widget_name)
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True
        self.update_filtered_df()

    def on_widget_change(self, change):
        # Update filtered DataFrame whenever a widget value changes
        self.update_filtered_df()

    def update_filtered_df(self):
        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        self.filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
        self.display_data_below(self.filtered_df)

    def filter_data(self, df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date):
        # Ensure that selected_level contains valid entries for filtering
        if selected_param == 'Level' and not selected_level:
            selected_level = df.columns.get_level_values('Level').unique()
        filtered_df = df.loc[start_date:end_date, \
                             (df.columns.get_level_values('UDL').isin(selected_udl)) &
                             (df.columns.get_level_values('Matu').isin(selected_matu)) &
                             (df.columns.get_level_values('Param').isin([selected_type])) &
                             (df.columns.get_level_values('Level').isin(selected_level))]
        return filtered_df

    def on_plot_button_clicked(self, b):
        plot_type = self.widgets.plot_type_widget.value
        plot_function = self.plot_manager.get_plot_function(plot_type)

        # Handle None plot function (invalid plot type)
        if plot_function is None:
            with self.output_plot:
                clear_output(wait=True)
                print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
            return

        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        window = self.widgets.window_widget.value
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        # Track input changes
        self.input_changes_dict = {
            "Plot Type": plot_type,
            "UDL": selected_udl,
            "Maturity": selected_matu,
            "Type": selected_type,
            "Param": selected_param,
            "Level": selected_level,
            "Window": window,
            "Start Date": start_date,
            "End Date": end_date
        }

        # Display debug table below
        with self.df_below_output:
            clear_output(wait=True)
            debug_df = pd.DataFrame(self.input_changes_dict.items(), columns=["Parameter", "Value"])
            display(HTML(debug_df.to_html(index=False)))

        if start_date > end_date:
            with self.output_plot:
                clear_output(wait=True)
                print("Start Date must be before End Date. Please correct your selection.")
            return

        try:
            filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
            if filtered_df.empty:
                with self.output_plot:
                    clear_output(wait=True)
                    print("No valid data available for the selected combination. Please adjust your selection.")
                    self.widgets.add_to_slide_button.disabled = True
                    return

            # Display the filtered DataFrame
            self.display_data_below(filtered_df)

            with self.main_display:
                clear_output(wait=True)
                if window and plot_type == "Time Series Plot":
                    fig = plot_function(filtered_df, window)
                else:
                    fig = plot_function(filtered_df)
                plt.show()
                self.current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": plot_type,
                    "position": self.widgets.position_dropdown.value
                }
                self.widgets.add_to_slide_button.disabled = False
        except Exception as e:
            with self.output_plot:
                clear_output(wait=True)
                print(f"An error occurred while generating the plot: {e}")
                logging.error("Error in on_plot_button_clicked", exc_info=True)

    def on_add_to_slide_button_clicked(self, b):
        if not self.current_plot_data:
            return
        
        # Re-fetch the currently selected position to ensure the latest value is used
        position = self.widgets.position_dropdown.value
        self.current_plot_data["position"] = position
        
        slide = self.slide_manager.get_current_slide()
        
        # Validate position is not already occupied
        if any(plot["position"] == position for plot in slide["plots"]):
            with self.main_display:
                clear_output(wait=True)
                print(f"Position '{position}' is already occupied. Please choose another position.")
                return

        # Add the plot to the slide if position is available
        self.slide_manager.add_plot_to_slide(self.current_plot_data.copy(), position)
        self.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
        self.switch_to_slide_view()
        self.update_sidebar()

    def on_add_slide_button_clicked(self, b):
        self.slide_manager.add_slide()
        self.update_sidebar()
        self.switch_to_slide_view()

    def on_export_button_clicked(self, b):
        prs = Presentation()
        for slide in self.slide_manager.slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig = plot_data["figure"]
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)
                left, top = self.get_slide_position(plot_data["position"])
                slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        pptx_filename = "Generated_Presentation.pptx"
        prs.save(pptx_filename)
        logging.info(f"Presentation exported as '{pptx_filename}'")
        
        # Ensure file visibility in Jupyter environment
        if os.path.exists(pptx_filename):
            print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
        else:
            print("Error: The presentation could not be saved.")

    def get_slide_position(self, position):
        if position == "top-left":
            return Inches(0), Inches(0)
        elif position == "top-right":
            return Inches(5), Inches(0)
        elif position == "bottom-left":
            return Inches(0), Inches(3.75)
        elif position == "bottom-right":
            return Inches(5), Inches(3.75)
        return Inches(0), Inches(0)

    def switch_to_slide_view(self):
        self.main_display.clear_output(wait=True)
        with self.main_display:
            slide = self.slide_manager.get_current_slide()
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)
                    ax.set_title(plot_data["type"])
                else:
                    logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

            plt.tight_layout()
            plt.show()

    def update_sidebar(self):
        sidebar_content = []
        for i, slide in enumerate(self.slide_manager.slides):
            fig, axs = plt.subplots(2, 2, figsize=(4, 4))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)

            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            plt.close(fig)
            buf.seek(0)
            img = Image.open(buf)
            img.thumbnail(THUMBNAIL_SIZE)

            with BytesIO() as output:
                img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
            slide_button_box.on_click(lambda b, idx=i: self.select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        self.left_sidebar.children = sidebar_content

    def select_slide(self, index):
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# APP - 2. COPY 2

In [4]:
# Step 7: Main app class
class App:
    def __init__(self, df):
        # Adjust df_below_output to tightly fit the main display without extra white space
        self.df_below_output = widgets.Output(layout=widgets.Layout(width="100%", border="1px solid #d3d3d3", padding="10px", margin="0 auto"))  # Adjusted output view for filtered DataFrame
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.register_plots()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="70%", height="500px", border="1px solid black", padding="0px", margin="0 auto"))  # Adjusted to match the width with df_below_output
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.input_changes_dict = {}  # Dictionary to track input changes
        self.filtered_df = df  # Initialize with the full DataFrame
        self.build_layout()
        self.bind_events()
        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def register_plots(self):
        self.plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
        self.plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "start_date_widget", "end_date_widget"])

        # Update plot_type_widget with available plot types
        self.widgets.plot_type_widget.options = list(self.plot_manager.plots.keys())
        self.widgets.plot_type_widget.value = list(self.plot_manager.plots.keys())[0]  # Set default value

    def build_layout(self):
        control_buttons = widgets.GridBox([
            self.widgets.add_slide_button, self.widgets.export_button,
            self.widgets.plot_button, self.widgets.add_to_slide_button
        ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
        
        self.customization_window = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
            self.widgets.plot_type_widget,
            self.widgets.udl_widget,
            self.widgets.type_widget,
            self.widgets.param_widget,
            self.widgets.level_widget,
            self.widgets.put_level_widget,
            self.widgets.call_level_widget,
            self.widgets.window_widget,
            self.widgets.start_date_widget,
            self.widgets.end_date_widget,
            self.widgets.position_dropdown,
            control_buttons,
        ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
        
        # Display customization window on the right, plot or slide view in the center
        display(widgets.VBox([
            widgets.HBox([self.left_sidebar, self.main_display, self.customization_window], layout=widgets.Layout(align_items='flex-start')),
            self.df_below_output  # Displaying the filtered DataFrame below
        ]))

    def display_data_below(self, df):
        # Display the filtered dataframe below the main view without clearing the existing content
        with self.df_below_output:
            clear_output(wait=True)
            display(HTML(df.to_html(max_rows=10)))  # Set the table width to "100%" so it fits within its container

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.type_widget.observe(self.on_type_change, names='value')
        self.widgets.param_widget.observe(self.on_param_change, names='value')
        self.widgets.matu_widget.observe(self.on_widget_change, names='value')
        self.widgets.udl_widget.observe(self.on_widget_change, names='value')
        self.widgets.level_widget.observe(self.on_widget_change, names='value')
        self.widgets.plot_button.on_click(self.on_plot_button_clicked)
        self.widgets.add_to_slide_button.on_click(self.on_add_to_slide_button_clicked)
        self.widgets.add_slide_button.on_click(self.on_add_slide_button_clicked)
        self.widgets.export_button.on_click(self.on_export_button_clicked)

    def on_type_change(self, change):
        selected_type = change['new']
        if selected_type == 'Spot':
            self.widgets.param_widget.options = ['Level', 'Returns']
            self.widgets.param_widget.value = 'Level'
            # Hide Maturity and Level-related widgets for Spot with Level selection
            self.widgets.matu_widget.layout.display = 'none'
            self.widgets.level_widget.layout.display = 'none'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
        elif selected_type in ['Moneyness', 'Delta']:
            self.widgets.param_widget.options = ['Level', 'Carry', 'Convex', 'Shift', 'Skew', 'Term Structure']
            self.widgets.param_widget.value = 'Level'
            # Show Maturity and Level-related widgets for Moneyness and Delta
            self.widgets.matu_widget.layout.display = 'flex'
            self.widgets.level_widget.layout.display = 'flex'
            self.widgets.put_level_widget.layout.display = 'none'
            self.widgets.call_level_widget.layout.display = 'none'
            
            # Update level options based on selected type
            if selected_type == 'Delta':
                self.widgets.level_widget.options = [str(delta) for delta in [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95]]
            elif selected_type == 'Moneyness':
                self.widgets.level_widget.options = [str(m) for m in self.df.columns.get_level_values('Level').unique()]
        
        # Hide/show widgets depending on selected param
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def on_param_change(self, change):
        # Update visibility of widgets when param changes
        self.update_param_widgets_visibility()
        self.update_filtered_df()

    def update_param_widgets_visibility(self):
        param_value = self.widgets.param_widget.value
        self.widgets.window_widget.layout.display = 'none'
        self.widgets.put_level_widget.layout.display = 'none'
        self.widgets.call_level_widget.layout.display = 'none'
        self.widgets.level_widget.layout.display = 'none'
        self.widgets.matu_widget.layout.display = 'none'

        if param_value == 'Returns':
            self.widgets.window_widget.layout.display = 'flex'
        elif param_value in ['Skew', 'Convex']:
            self.widgets.put_level_widget.layout.display = 'flex'
            self.widgets.call_level_widget.layout.display = 'flex'
        elif param_value == 'Level':
            if self.widgets.type_widget.value != 'Spot':
                self.widgets.matu_widget.layout.display = 'flex'
                self.widgets.level_widget.layout.display = 'flex'

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_manager.get_required_widgets(selected_plot)

        for widget_name in ["plot_type_widget", "udl_widget", "matu_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
            widget = getattr(self.widgets, widget_name)
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True
        self.update_filtered_df()

    def on_widget_change(self, change):
        # Update filtered DataFrame whenever a widget value changes
        self.update_filtered_df()

    def update_filtered_df(self):
        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        self.filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
        self.display_data_below(self.filtered_df)

    def filter_data(self, df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date):
        # Ensure that selected_level contains valid entries for filtering
        if selected_param == 'Level' and not selected_level:
            selected_level = df.columns.get_level_values('Level').unique()
        filtered_df = df.loc[start_date:end_date, \
                             (df.columns.get_level_values('UDL').isin(selected_udl)) &
                             (df.columns.get_level_values('Matu').isin(selected_matu)) &
                             (df.columns.get_level_values('Param').isin([selected_type])) &
                             (df.columns.get_level_values('Level').isin(selected_level))]
        return filtered_df

    def on_plot_button_clicked(self, b):
        plot_type = self.widgets.plot_type_widget.value
        plot_function = self.plot_manager.get_plot_function(plot_type)

        # Handle None plot function (invalid plot type)
        if plot_function is None:
            with self.output_plot:
                clear_output(wait=True)
                print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
            return

        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        window = self.widgets.window_widget.value
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        # Track input changes
        self.input_changes_dict = {
            "Plot Type": plot_type,
            "UDL": selected_udl,
            "Maturity": selected_matu,
            "Type": selected_type,
            "Param": selected_param,
            "Level": selected_level,
            "Window": window,
            "Start Date": start_date,
            "End Date": end_date
        }

        # Display debug table below
        with self.df_below_output:
            clear_output(wait=True)
            debug_df = pd.DataFrame(self.input_changes_dict.items(), columns=["Parameter", "Value"])
            display(HTML(debug_df.to_html(index=False)))

        if start_date > end_date:
            with self.output_plot:
                clear_output(wait=True)
                print("Start Date must be before End Date. Please correct your selection.")
            return

        try:
            filtered_df = self.filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
            if filtered_df.empty:
                with self.output_plot:
                    clear_output(wait=True)
                    print("No valid data available for the selected combination. Please adjust your selection.")
                    self.widgets.add_to_slide_button.disabled = True
                    return

            # Display the filtered DataFrame
            self.display_data_below(filtered_df)

            with self.main_display:
                clear_output(wait=True)
                if window and plot_type == "Time Series Plot":
                    fig = plot_function(filtered_df, window)
                else:
                    fig = plot_function(filtered_df)
                plt.show()
                self.current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": plot_type,
                    "position": self.widgets.position_dropdown.value
                }
                self.widgets.add_to_slide_button.disabled = False
        except Exception as e:
            with self.output_plot:
                clear_output(wait=True)
                print(f"An error occurred while generating the plot: {e}")
                logging.error("Error in on_plot_button_clicked", exc_info=True)

    def on_add_to_slide_button_clicked(self, b):
        if not self.current_plot_data:
            return
        
        # Re-fetch the currently selected position to ensure the latest value is used
        position = self.widgets.position_dropdown.value
        self.current_plot_data["position"] = position
        
        slide = self.slide_manager.get_current_slide()
        
        # Validate position is not already occupied
        if any(plot["position"] == position for plot in slide["plots"]):
            with self.main_display:
                clear_output(wait=True)
                print(f"Position '{position}' is already occupied. Please choose another position.")
                return

        # Add the plot to the slide if position is available
        self.slide_manager.add_plot_to_slide(self.current_plot_data.copy(), position)
        self.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
        self.switch_to_slide_view()
        self.update_sidebar()

    def on_add_slide_button_clicked(self, b):
        self.slide_manager.add_slide()
        self.update_sidebar()
        self.switch_to_slide_view()

    def on_export_button_clicked(self, b):
        prs = Presentation()
        for slide in self.slide_manager.slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig = plot_data["figure"]
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)
                left, top = self.get_slide_position(plot_data["position"])
                slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        pptx_filename = "Generated_Presentation.pptx"
        prs.save(pptx_filename)
        logging.info(f"Presentation exported as '{pptx_filename}'")
        
        # Ensure file visibility in Jupyter environment
        if os.path.exists(pptx_filename):
            print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
        else:
            print("Error: The presentation could not be saved.")

    def get_slide_position(self, position):
        if position == "top-left":
            return Inches(0), Inches(0)
        elif position == "top-right":
            return Inches(5), Inches(0)
        elif position == "bottom-left":
            return Inches(0), Inches(3.75)
        elif position == "bottom-right":
            return Inches(5), Inches(3.75)
        return Inches(0), Inches(0)

    def switch_to_slide_view(self):
        self.main_display.clear_output(wait=True)
        with self.main_display:
            slide = self.slide_manager.get_current_slide()
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)
                    ax.set_title(plot_data["type"])
                else:
                    logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

            plt.tight_layout()
            plt.show()

    def update_sidebar(self):
        sidebar_content = []
        for i, slide in enumerate(self.slide_manager.slides):
            fig, axs = plt.subplots(2, 2, figsize=(4, 4))
            for ax in axs.flatten():
                ax.clear()
                ax.axis('off')  # Initially hide all axes

            for plot_data in slide["plots"]:
                pos = POSITION_MAP.get(plot_data["position"], None)
                if pos is not None:
                    ax = axs[pos]
                    ax.axis('on')  # Enable axis for valid plots
                    generate_plot_image(plot_data["figure"], ax)

            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            plt.close(fig)
            buf.seek(0)
            img = Image.open(buf)
            img.thumbnail(THUMBNAIL_SIZE)

            with BytesIO() as output:
                img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
            slide_button_box.on_click(lambda b, idx=i: self.select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        self.left_sidebar.children = sidebar_content

    def select_slide(self, index):
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# APP 3. CREATE - Extracted the methods from the App class and refactored the class to use these standalone functions. 
The functions are now modular and should be easier to debug and extend. If you encounter any specific issues or have questions about the refactoring process, feel free to ask!

## Functions defined outside the class

In [ ]:
def register_plots(plot_manager, widgets):
    plot_manager.register_plot("Time Series Plot", create_time_series_plot, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"])
    plot_manager.register_plot("Custom Stats Chart", create_stats_chart, ["plot_type_widget", "udl_widget", "type_widget", "param_widget", "level_widget", "put_level_widget", "call_level_widget", "start_date_widget", "end_date_widget"])
    
    # Update plot_type_widget with available plot types
    widgets.plot_type_widget.options = list(plot_manager.plots.keys())
    widgets.plot_type_widget.value = list(plot_manager.plots.keys())[0]  # Set default value


def build_layout(app):
    control_buttons = widgets.GridBox([
        app.widgets.add_slide_button, app.widgets.export_button,
        app.widgets.plot_button, app.widgets.add_to_slide_button
    ], layout=widgets.Layout(grid_template_columns="repeat(2, 48%)", grid_gap="5px", width="100%", min_height="60px"))
    
    app.customization_window = widgets.VBox([
        widgets.HTML(value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"),
        app.widgets.plot_type_widget,
        app.widgets.udl_widget,
        app.widgets.type_widget,
        app.widgets.param_widget,
        app.widgets.level_widget,
        app.widgets.put_level_widget,
        app.widgets.call_level_widget,
        app.widgets.window_widget,
        app.widgets.start_date_widget,
        app.widgets.end_date_widget,
        app.widgets.position_dropdown,
        control_buttons,
    ], layout=widgets.Layout(width="30%", padding="15px", border="1px solid #d3d3d3", background_color="#f9f9f9"))
    
    # Display customization window on the right, plot or slide view in the center
    display(widgets.VBox([
        widgets.HBox([app.left_sidebar, app.main_display, app.customization_window], layout=widgets.Layout(align_items='flex-start')),
        app.df_below_output  # Displaying the filtered DataFrame below
    ]))


def display_data_below(df_below_output, df):
    # Display the filtered dataframe below the main view without clearing the existing content
    with df_below_output:
        clear_output(wait=True)
        display(HTML(df.to_html(max_rows=10)))  # Set the table width to "100%" so it fits within its container


def bind_events(app):
    app.widgets.plot_type_widget.observe(lambda change: on_plot_type_change(app, change), names='value')
    app.widgets.type_widget.observe(lambda change: on_type_change(app, change), names='value')
    app.widgets.param_widget.observe(lambda change: on_param_change(app, change), names='value')
    app.widgets.matu_widget.observe(lambda change: on_widget_change(app, change), names='value')
    app.widgets.udl_widget.observe(lambda change: on_widget_change(app, change), names='value')
    app.widgets.level_widget.observe(lambda change: on_widget_change(app, change), names='value')
    app.widgets.plot_button.on_click(lambda b: on_plot_button_clicked(app, b))
    app.widgets.add_to_slide_button.on_click(lambda b: on_add_to_slide_button_clicked(app, b))
    app.widgets.add_slide_button.on_click(lambda b: on_add_slide_button_clicked(app, b))
    app.widgets.export_button.on_click(lambda b: on_export_button_clicked(app, b))


def on_type_change(app, change):
    selected_type = change['new']
    if selected_type == 'Spot':
        app.widgets.param_widget.options = ['Level', 'Returns']
        app.widgets.param_widget.value = 'Level'
        # Hide Maturity and Level-related widgets for Spot with Level selection
        app.widgets.matu_widget.layout.display = 'none'
        app.widgets.level_widget.layout.display = 'none'
        app.widgets.put_level_widget.layout.display = 'none'
        app.widgets.call_level_widget.layout.display = 'none'
    elif selected_type in ['Moneyness', 'Delta']:
        app.widgets.param_widget.options = ['Level', 'Carry', 'Convex', 'Shift', 'Skew', 'Term Structure']
        app.widgets.param_widget.value = 'Level'
        # Show Maturity and Level-related widgets for Moneyness and Delta
        app.widgets.matu_widget.layout.display = 'flex'
        app.widgets.level_widget.layout.display = 'flex'
        app.widgets.put_level_widget.layout.display = 'none'
        app.widgets.call_level_widget.layout.display = 'none'
        
        # Update level options based on selected type
        if selected_type == 'Delta':
            app.widgets.level_widget.options = [str(delta) for delta in [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95]]
        elif selected_type == 'Moneyness':
            app.widgets.level_widget.options = [str(m) for m in app.df.columns.get_level_values('Level').unique()]
    
    # Hide/show widgets depending on selected param
    app.update_param_widgets_visibility()
    app.update_filtered_df()


def update_param_widgets_visibility(app):
    param_value = app.widgets.param_widget.value
    app.widgets.window_widget.layout.display = 'none'
    app.widgets.put_level_widget.layout.display = 'none'
    app.widgets.call_level_widget.layout.display = 'none'
    app.widgets.level_widget.layout.display = 'none'
    app.widgets.matu_widget.layout.display = 'none'

    if param_value == 'Returns':
        app.widgets.window_widget.layout.display = 'flex'
    elif param_value in ['Skew', 'Convex']:
        app.widgets.put_level_widget.layout.display = 'flex'
        app.widgets.call_level_widget.layout.display = 'flex'
    elif param_value == 'Level':
        if app.widgets.type_widget.value != 'Spot':
            app.widgets.matu_widget.layout.display = 'flex'
            app.widgets.level_widget.layout.display = 'flex'


def on_widget_change(app, change):
    # Update filtered DataFrame whenever a widget value changes
    app.update_filtered_df()


def filter_data(df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date):
    # Ensure that selected_level contains valid entries for filtering
    if selected_param == 'Level' and not selected_level:
        selected_level = df.columns.get_level_values('Level').unique()
    filtered_df = df.loc[start_date:end_date, \
                         (df.columns.get_level_values('UDL').isin(selected_udl)) &
                         (df.columns.get_level_values('Matu').isin(selected_matu)) &
                         (df.columns.get_level_values('Param').isin([selected_type])) &
                         (df.columns.get_level_values('Level').isin(selected_level))]
    return filtered_df


def on_plot_button_clicked(app, b):
    plot_type = app.widgets.plot_type_widget.value
    plot_function = app.plot_manager.get_plot_function(plot_type)

    # Handle None plot function (invalid plot type)
    if plot_function is None:
        with app.output_plot:
            clear_output(wait=True)
            print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
        return

    selected_udl = list(app.widgets.udl_widget.value)
    selected_matu = list(app.widgets.matu_widget.value) if app.widgets.matu_widget.layout.display != 'none' else []
    selected_type = app.widgets.type_widget.value
    selected_param = app.widgets.param_widget.value
    selected_level = list(app.widgets.level_widget.value) if app.widgets.level_widget.layout.display != 'none' else []
    window = app.widgets.window_widget.value
    start_date = app.widgets.start_date_widget.value
    end_date = app.widgets.end_date_widget.value

    # Track input changes
    app.input_changes_dict = {
        "Plot Type": plot_type,
        "UDL": selected_udl,
        "Maturity": selected_matu,
        "Type": selected_type,
        "Param": selected_param,
        "Level": selected_level,
        "Window": window,
        "Start Date": start_date,
        "End Date": end_date
    }

    # Display debug table below
    with app.df_below_output:
        clear_output(wait=True)
        debug_df = pd.DataFrame(app.input_changes_dict.items(), columns=["Parameter", "Value"])
        display(HTML(debug_df.to_html(index=False)))

    if start_date > end_date:
        with app.output_plot:
            clear_output(wait=True)
            print("Start Date must be before End Date. Please correct your selection.")
        return

    try:
        filtered_df = filter_data(app.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
        if filtered_df.empty:
            with app.output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                app.widgets.add_to_slide_button.disabled = True
                return

        # Display the filtered DataFrame
        app.display_data_below(filtered_df)

        with app.main_display:
            clear_output(wait=True)
            if window and plot_type == "Time Series Plot":
                fig = plot_function(filtered_df, window)
            else:
                fig = plot_function(filtered_df)
            plt.show()
            app.current_plot_data = {
                "figure": fig,
                "data": filtered_df,
                "type": plot_type,
                "position": app.widgets.position_dropdown.value
            }
            app.widgets.add_to_slide_button.disabled = False
    except Exception as e:
        with app.output_plot:
            clear_output(wait=True)
            print(f"An error occurred while generating the plot: {e}")
            logging.error("Error in on_plot_button_clicked", exc_info=True)


def on_add_to_slide_button_clicked(app, b):
    if not app.current_plot_data:
        return
    
    # Re-fetch the currently selected position to ensure the latest value is used
    position = app.widgets.position_dropdown.value
    app.current_plot_data["position"] = position
    
    slide = app.slide_manager.get_current_slide()
    
    # Validate position is not already occupied
    if any(plot["position"] == position for plot in slide["plots"]):
        with app.main_display:
            clear_output(wait=True)
            print(f"Position '{position}' is already occupied. Please choose another position.")
            return

    # Add the plot to the slide if position is available
    app.slide_manager.add_plot_to_slide(app.current_plot_data.copy(), position)
    app.current_plot_data = {}  # Clear the current plot data to prevent duplicate additions
    app.switch_to_slide_view()
    app.update_sidebar()


def on_add_slide_button_clicked(app, b):
    app.slide_manager.add_slide()
    app.update_sidebar()
    app.switch_to_slide_view()


def on_export_button_clicked(app, b):
    prs = Presentation()
    for slide in app.slide_manager.slides:
        slide_layout = prs.slide_layouts[5]
        slide_to_add = prs.slides.add_slide(slide_layout)

        for plot_data in slide["plots"]:
            img_stream = BytesIO()
            fig = plot_data["figure"]
            fig.savefig(img_stream, format='png')
            img_stream.seek(0)
            left, top = app.get_slide_position(plot_data["position"])
            slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
            plt.close(fig)

    pptx_filename = "Generated_Presentation.pptx"
    prs.save(pptx_filename)
    logging.info(f"Presentation exported as '{pptx_filename}'")
    
    # Ensure file visibility in Jupyter environment
    if os.path.exists(pptx_filename):
        print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
    else:
        print("Error: The presentation could not be saved.")


def get_slide_position(position):
    if position == "top-left":
        return Inches(0), Inches(0)
    elif position == "top-right":
        return Inches(5), Inches(0)
    elif position == "bottom-left":
        return Inches(0), Inches(3.75)
    elif position == "bottom-right":
        return Inches(5), Inches(3.75)
    return Inches(0), Inches(0)


def switch_to_slide_view(app):
    app.main_display.clear_output(wait=True)
    with app.main_display:
        slide = app.slide_manager.get_current_slide()
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        for ax in axs.flatten():
            ax.clear()
            ax.axis('off')  # Initially hide all axes

        for plot_data in slide["plots"]:
            pos = POSITION_MAP.get(plot_data["position"], None)
            if pos is not None:
                ax = axs[pos]
                ax.axis('on')  # Enable axis for valid plots
                generate_plot_image(plot_data["figure"], ax)
                ax.set_title(plot_data["type"])
            else:
                logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

        plt.tight_layout()
        plt.show()


def update_sidebar(app):
    sidebar_content = []
    for i, slide in enumerate(app.slide_manager.slides):
        fig, axs = plt.subplots(2, 2, figsize=(4, 4))
        for ax in axs.flatten():
            ax.clear()
            ax.axis('off')  # Initially hide all axes

        for plot_data in slide["plots"]:
            pos = POSITION_MAP.get(plot_data["position"], None)
            if pos is not None:
                ax = axs[pos]
                ax.axis('on')  # Enable axis for valid plots
                generate_plot_image(plot_data["figure"], ax)

        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail(THUMBNAIL_SIZE)

        with BytesIO() as output:
            img.save(output, format="PNG")
            img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

        slide_label = widgets.Label(f"Slide {i + 1}")
        slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
            width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
        ))

        slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
        slide_button_box.on_click(lambda b, idx=i: app.select_slide(idx))

        sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

    app.left_sidebar.children = sidebar_content


def select_slide(app, index):
    app.slide_manager.selected_slide_index = index
    app.switch_to_slide_view()


# Main app class
class App:
    def __init__(self, df):
        # Adjust df_below_output to tightly fit the main display without extra white space
        self.df_below_output = widgets.Output(layout=widgets.Layout(width="100%", border="1px solid #d3d3d3", padding="10px", margin="0 auto"))  # Adjusted output view for filtered DataFrame
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}
        self.output_plot = widgets.Output()
        self.main_display = widgets.Output(layout=widgets.Layout(width="70%", height="500px", border="1px solid black", padding="0px", margin="0 auto"))  # Adjusted to match the width with df_below_output
        self.left_sidebar = widgets.VBox(layout=widgets.Layout(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0"))
        self.input_changes_dict = {}  # Dictionary to track input changes
        self.filtered_df = df  # Initialize with the full DataFrame

        # Call external functions
        register_plots(self.plot_manager, self.widgets)
        build_layout(self)
        bind_events(self)

        self.update_sidebar()
        self.run_app()

    def run_app(self):
        display(self)

    def display_data_below(self, df):
        display_data_below(self.df_below_output, df)

    def update_param_widgets_visibility(self):
        update_param_widgets_visibility(self)

    def update_filtered_df(self):
        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = list(self.widgets.matu_widget.value) if self.widgets.matu_widget.layout.display != 'none' else []
        selected_type = self.widgets.type_widget.value
        selected_param = self.widgets.param_widget.value
        selected_level = list(self.widgets.level_widget.value) if self.widgets.level_widget.layout.display != 'none' else []
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        self.filtered_df = filter_data(self.df, selected_udl, selected_matu, selected_type, selected_param, selected_level, start_date, end_date)
        self.display_data_below(self.filtered_df)

    def get_slide_position(self, position):
        return get_slide_position(position)

    def switch_to_slide_view(self):
        switch_to_slide_view(self)

    def update_sidebar(self):
        update_sidebar(self)

    def select_slide(self, index):
        select_slide(self, index)


# Inputs

In [3]:
# Configuration constants
FIGURE_SIZE = (5, 4)
THUMBNAIL_SIZE = (120, 120)
POSITION_MAP = {
    "top-left": (0, 0),
    "top-right": (0, 1),
    "bottom-left": (1, 0),
    "bottom-right": (1, 1)
}

# Setup logging
logging.basicConfig(level=logging.INFO)

# Step 8: Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': [],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
}

# Display

In [4]:
df = create_multi_index_dataframe(udl, matu, param, level)
app = App(df)
display(app)